In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/yellow_tripdata_2022_sample.csv')  # sampling data
df_taxi_zone = pd.read_csv('data/taxi_zones.csv')
print(df.shape)
df.head()

(19200, 19)


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2022-01-20 19:12:17,2022-01-20 19:16:35,1.0,0.67,1.0,N,137,233,1,5.0,1.00,0.5,1.86,0.00,0.3,11.16,2.5,0.00
1,1,2022-01-02 19:16:15,2022-01-02 19:23:56,1.0,2.30,1.0,N,140,170,1,9.5,2.50,0.5,3.20,0.00,0.3,16.00,2.5,0.00
2,1,2022-01-08 18:23:24,2022-01-08 18:56:33,1.0,17.20,2.0,N,132,107,1,52.0,3.75,0.5,12.62,6.55,0.3,75.72,2.5,1.25
3,2,2022-01-14 09:30:03,2022-01-14 09:30:22,2.0,0.05,1.0,N,79,79,2,2.5,0.00,0.5,0.00,0.00,0.3,5.80,2.5,0.00
4,1,2022-01-18 12:40:31,2022-01-18 12:47:19,0.0,0.80,1.0,N,107,170,2,6.5,2.50,0.5,0.00,0.00,0.3,9.80,2.5,0.00


In [3]:
def get_coordinates(id, column='x'):
    try:
        return df_taxi_zone[df_taxi_zone['location_id'] == id][[f'coordinate_{column}']].values[0][0]
    except IndexError:
        return 0

In [4]:
get_coordinates(3), get_coordinates(2024)  #Testing Function

(-73.84742223236718, 0)

In [5]:
df.dropna(inplace=True)

In [6]:
df.shape

(18546, 19)

In [7]:
df_taxi_zone

,location_id,zone,coordinate_x,coordinate_y
0,1,Newark Airport,-74.174000,40.691831
1,2,Jamaica Bay,-73.831299,40.616745
2,3,Allerton/Pelham Gardens,-73.847422,40.864474
3,4,Alphabet City,-73.976968,40.723752
4,5,Arden Heights,-74.188484,40.552659
...,...,...,...,...
258,256,Williamsburg (South Side),-73.959905,40.710880
259,259,Woodlawn/Wakefield,-73.852215,40.897932
260,260,Woodside,-73.906306,40.744235
261,261,World Trade Center,-74.013023,40.709139


In [8]:
df["PULocation_x"]= df.apply(lambda row: get_coordinates(row["PULocationID"]), axis=1)
print('PULocation_x done')
df["PULocation_y"]= df.apply(lambda row: get_coordinates(row["PULocationID"], 'y'), axis=1)
print('PULocation_y done')
df["DOLocation_x"]= df.apply(lambda row: get_coordinates(row["DOLocationID"]), axis=1)
print('DOLocation_x done')
df["DOLocation_y"]= df.apply(lambda row: get_coordinates(row["PULocationID"],'y'), axis=1)
print('DOLocation_y done')

df.drop(['PULocationID', 'DOLocationID'], axis=1, inplace=True)

PULocation_x done
PULocation_y done
DOLocation_x done
DOLocation_y done


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18546 entries, 0 to 19199
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   VendorID               18546 non-null  int64  
 1   tpep_pickup_datetime   18546 non-null  object 
 2   tpep_dropoff_datetime  18546 non-null  object 
 3   passenger_count        18546 non-null  float64
 4   trip_distance          18546 non-null  float64
 5   RatecodeID             18546 non-null  float64
 6   store_and_fwd_flag     18546 non-null  object 
 7   payment_type           18546 non-null  int64  
 8   fare_amount            18546 non-null  float64
 9   extra                  18546 non-null  float64
 10  mta_tax                18546 non-null  float64
 11  tip_amount             18546 non-null  float64
 12  tolls_amount           18546 non-null  float64
 13  improvement_surcharge  18546 non-null  float64
 14  total_amount           18546 non-null  float64
 15  congest

In [10]:
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18546 entries, 0 to 19199
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   VendorID               18546 non-null  int64         
 1   tpep_pickup_datetime   18546 non-null  datetime64[ns]
 2   tpep_dropoff_datetime  18546 non-null  datetime64[ns]
 3   passenger_count        18546 non-null  float64       
 4   trip_distance          18546 non-null  float64       
 5   RatecodeID             18546 non-null  float64       
 6   store_and_fwd_flag     18546 non-null  object        
 7   payment_type           18546 non-null  int64         
 8   fare_amount            18546 non-null  float64       
 9   extra                  18546 non-null  float64       
 10  mta_tax                18546 non-null  float64       
 11  tip_amount             18546 non-null  float64       
 12  tolls_amount           18546 non-null  float64       
 13  improv

In [12]:
df = df.drop_duplicates().reset_index(drop=True)
df['trip_id'] = df.index

In [13]:
datetime_dim = df[['tpep_pickup_datetime', 'tpep_dropoff_datetime']].reset_index(drop=True)
datetime_dim['pick_hour'] = datetime_dim['tpep_pickup_datetime'].dt.hour
datetime_dim['pick_day'] = datetime_dim['tpep_pickup_datetime'].dt.day
datetime_dim['pick_month'] = datetime_dim['tpep_pickup_datetime'].dt.month
datetime_dim['pick_year'] = datetime_dim['tpep_pickup_datetime'].dt.year
datetime_dim['pick_weekday'] = datetime_dim['tpep_pickup_datetime'].dt.weekday


datetime_dim['drop_hour'] = datetime_dim['tpep_pickup_datetime'].dt.hour
datetime_dim['drop_day'] = datetime_dim['tpep_pickup_datetime'].dt.day
datetime_dim['drop_month'] = datetime_dim['tpep_pickup_datetime'].dt.month
datetime_dim['drop_year'] = datetime_dim['tpep_pickup_datetime'].dt.year
datetime_dim['drop_weekday'] = datetime_dim['tpep_pickup_datetime'].dt.weekday

datetime_dim

,tpep_pickup_datetime,tpep_dropoff_datetime,pick_hour,pick_day,pick_month,pick_year,pick_weekday,drop_hour,drop_day,drop_month,drop_year,drop_weekday
0,2022-01-20 19:12:17,2022-01-20 19:16:35,19,20,1,2022,3,19,20,1,2022,3
1,2022-01-02 19:16:15,2022-01-02 19:23:56,19,2,1,2022,6,19,2,1,2022,6
2,2022-01-08 18:23:24,2022-01-08 18:56:33,18,8,1,2022,5,18,8,1,2022,5
3,2022-01-14 09:30:03,2022-01-14 09:30:22,9,14,1,2022,4,9,14,1,2022,4
4,2022-01-18 12:40:31,2022-01-18 12:47:19,12,18,1,2022,1,12,18,1,2022,1
...,...,...,...,...,...,...,...,...,...,...,...,...
18541,2022-12-14 21:37:24,2022-12-14 21:59:17,21,14,12,2022,2,21,14,12,2022,2
18542,2022-12-23 14:45:08,2022-12-23 15:04:14,14,23,12,2022,4,14,23,12,2022,4
18543,2022-12-23 15:54:34,2022-12-23 15:59:58,15,23,12,2022,4,15,23,12,2022,4
18544,2022-12-20 01:31:22,2022-12-20 01:48:56,1,20,12,2022,1,1,20,12,2022,1


In [14]:
datetime_dim['datetime_id'] = datetime_dim.index

In [15]:
datetime_dim = datetime_dim[['datetime_id', 'tpep_pickup_datetime', 'pick_hour', 'pick_day', 'pick_month', 'pick_year', 'pick_weekday',
       'tpep_dropoff_datetime', 'drop_hour', 'drop_day', 'drop_month', 'drop_year', 'drop_weekday']]

In [16]:
datetime_dim

,datetime_id,tpep_pickup_datetime,pick_hour,pick_day,pick_month,pick_year,pick_weekday,tpep_dropoff_datetime,drop_hour,drop_day,drop_month,drop_year,drop_weekday
0,0,2022-01-20 19:12:17,19,20,1,2022,3,2022-01-20 19:16:35,19,20,1,2022,3
1,1,2022-01-02 19:16:15,19,2,1,2022,6,2022-01-02 19:23:56,19,2,1,2022,6
2,2,2022-01-08 18:23:24,18,8,1,2022,5,2022-01-08 18:56:33,18,8,1,2022,5
3,3,2022-01-14 09:30:03,9,14,1,2022,4,2022-01-14 09:30:22,9,14,1,2022,4
4,4,2022-01-18 12:40:31,12,18,1,2022,1,2022-01-18 12:47:19,12,18,1,2022,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18541,18541,2022-12-14 21:37:24,21,14,12,2022,2,2022-12-14 21:59:17,21,14,12,2022,2
18542,18542,2022-12-23 14:45:08,14,23,12,2022,4,2022-12-23 15:04:14,14,23,12,2022,4
18543,18543,2022-12-23 15:54:34,15,23,12,2022,4,2022-12-23 15:59:58,15,23,12,2022,4
18544,18544,2022-12-20 01:31:22,1,20,12,2022,1,2022-12-20 01:48:56,1,20,12,2022,1


In [17]:
passenger_count_dim = df[['passenger_count']].reset_index(drop=True)
passenger_count_dim['passenger_count_id'] = passenger_count_dim.index
passenger_count_dim['passenger_count'] = passenger_count_dim['passenger_count'].astype('int')
passenger_count_dim[['passenger_count_id', 'passenger_count']]

,passenger_count_id,passenger_count
0,0,1
1,1,1
2,2,1
3,3,2
4,4,0
...,...,...
18541,18541,1
18542,18542,6
18543,18543,1
18544,18544,1


In [18]:
trip_distance_dim = df[['trip_distance']].reset_index(drop=True)
trip_distance_dim['trip_distance_id'] = trip_distance_dim.index
trip_distance_dim[['trip_distance_id', 'trip_distance']]

,trip_distance_id,trip_distance
0,0,0.67
1,1,2.30
2,2,17.20
3,3,0.05
4,4,0.80
...,...,...
18541,18541,1.45
18542,18542,4.11
18543,18543,0.65
18544,18544,5.30


In [19]:
PU_location_dim = df[['PULocation_x', 'PULocation_y']].reset_index(drop=True)
PU_location_dim['PU_location_id'] = PU_location_dim.index
PU_location_dim[['PU_location_id', 'PULocation_x', 'PULocation_y']]

,PU_location_id,PULocation_x,PULocation_y
0,0,-73.976495,40.740439
1,1,-73.954739,40.765484
2,2,-73.786533,40.646985
3,3,-73.985937,40.727620
4,4,-73.984052,40.736824
...,...,...,...
18541,18541,-73.972356,40.756688
18542,18542,-73.978632,40.783961
18543,18543,-73.965635,40.768615
18544,18544,-73.990458,40.740337


In [20]:
DO_location_dim = df[['DOLocation_x', 'DOLocation_y']].reset_index(drop=True)
DO_location_dim['DO_location_id'] = DO_location_dim.index
DO_location_dim[['DO_location_id', 'DOLocation_x', 'DOLocation_y']]

,DO_location_id,DOLocation_x,DOLocation_y
0,0,-73.970443,40.740439
1,1,-73.978492,40.765484
2,2,-73.984052,40.646985
3,3,-73.985937,40.727620
4,4,-73.978492,40.736824
...,...,...,...
18541,18541,-73.992438,40.756688
18542,18542,-73.948522,40.783961
18543,18543,-73.959635,40.768615
18544,18544,-73.937346,40.740337


In [21]:
payment_type_dim = df[['payment_type']].reset_index(drop=True)
payment_type_dim['payment_type_id'] = payment_type_dim.index
payment_name_dict = {1: 'Credit card', 2: 'Cash', 3: 'No charge', 4: 'Dispute', 5: 'Unknown', 6: 'Voided trip'}
payment_type_dim['payment_type_name'] = payment_type_dim['payment_type'].map(payment_name_dict)
payment_type_dim[['payment_type_id', 'payment_type', 'payment_type_name']]

,payment_type_id,payment_type,payment_type_name
0,0,1,Credit card
1,1,1,Credit card
2,2,1,Credit card
3,3,2,Cash
4,4,2,Cash
...,...,...,...
18541,18541,2,Cash
18542,18542,1,Credit card
18543,18543,1,Credit card
18544,18544,3,No charge


In [22]:
rate_code_dim = df[['RatecodeID']].reset_index(drop=True)
rate_code_dim['rate_code_id'] = rate_code_dim.index
rate_code_name_dict = {1: 'Standard rate', 2: 'JFK', 3: 'Newark', 4: 'Nassau or Westchester', 5: 'Negotiated fare', 6: 'Group ride'}
rate_code_dim['rate_code_name'] = rate_code_dim['RatecodeID'].map(rate_code_name_dict)
rate_code_dim[['rate_code_id', 'RatecodeID', 'rate_code_name']]

,rate_code_id,RatecodeID,rate_code_name
0,0,1.0,Standard rate
1,1,1.0,Standard rate
2,2,2.0,JFK
3,3,1.0,Standard rate
4,4,1.0,Standard rate
...,...,...,...
18541,18541,1.0,Standard rate
18542,18542,1.0,Standard rate
18543,18543,1.0,Standard rate
18544,18544,1.0,Standard rate


In [23]:
fact_table = df.merge(passenger_count_dim, left_on='trip_id', right_on='passenger_count_id').merge(trip_distance_dim, left_on='trip_id', right_on='trip_distance_id').merge(rate_code_dim, left_on='trip_id', right_on='rate_code_id').merge(PU_location_dim, left_on='trip_id', right_on='PU_location_id').merge(DO_location_dim, left_on='trip_id', right_on='DO_location_id').merge(datetime_dim, left_on='trip_id', right_on='datetime_id').merge(payment_type_dim, left_on='trip_id', right_on='payment_type_id')[['trip_id','VendorID', 'datetime_id', 'passenger_count_id',
               'trip_distance_id', 'rate_code_id', 'store_and_fwd_flag', 'PU_location_id', 'DO_location_id',
               'payment_type_id', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
               'improvement_surcharge', 'total_amount']]

In [24]:
fact_table

,trip_id,VendorID,datetime_id,passenger_count_id,trip_distance_id,rate_code_id,store_and_fwd_flag,PU_location_id,DO_location_id,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,0,2,0,0,0,0,N,0,0,0,5.0,1.00,0.5,1.86,0.00,0.3,11.16
1,1,1,1,1,1,1,N,1,1,1,9.5,2.50,0.5,3.20,0.00,0.3,16.00
2,2,1,2,2,2,2,N,2,2,2,52.0,3.75,0.5,12.62,6.55,0.3,75.72
3,3,2,3,3,3,3,N,3,3,3,2.5,0.00,0.5,0.00,0.00,0.3,5.80
4,4,1,4,4,4,4,N,4,4,4,6.5,2.50,0.5,0.00,0.00,0.3,9.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18541,18541,2,18541,18541,18541,18541,N,18541,18541,18541,14.5,0.50,0.5,0.00,0.00,0.3,18.30
18542,18542,2,18542,18542,18542,18542,N,18542,18542,18542,22.6,0.00,0.5,6.65,0.00,1.0,33.25
18543,18543,2,18543,18543,18543,18543,N,18543,18543,18543,7.2,0.00,0.5,2.00,0.00,1.0,13.20
18544,18544,1,18544,18544,18544,18544,N,18544,18544,18544,23.3,3.50,0.5,0.00,0.00,1.0,28.30
